In [29]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np


In [ ]:
%cd /content/drive/MyDrive/RohithWorkspace/Object-Detection-Website
!git add -A
!git commit -m "Still testing google colab"
!git push